# Moodify - Text Emotion Recognition

In [ ]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from IPython.display import display
from transformers import AutoModelForSequenceClassification, AutoTokenizer

In [ ]:
textarea = widgets.Textarea(
    value="I am so happy today!",
    placeholder="Insert text here",
    description="Text:",
    disabled=False,
)

display(textarea)

In [ ]:
# model_name = "j-hartmann/emotion-english-distilroberta-base"
model_name = "michellejieli/emotion_text_classifier"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

inputs = tokenizer(textarea.value, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs)

probs = torch.nn.functional.softmax(outputs.logits, dim=-1).squeeze()
emotion_labels = model.config.id2label
emotion_scores = {emotion_labels[i]: probs[i].item() for i in range(len(probs))}
emotion_scores = dict(
    sorted(emotion_scores.items(), key=lambda item: item[1], reverse=True)
)

emotions = list(emotion_scores.keys())
scores = list(emotion_scores.values())

plt.figure(figsize=(8, 4))
sns.barplot(x=emotions, y=scores, palette="viridis", hue=emotions)
plt.xlabel("Emotion")
plt.ylabel("Confidence Score")
plt.title("Emotion Predictions")
plt.ylim(0, 1)
plt.show()

print("\nEmotion Predictions:\n")
for emotion, score in emotion_scores.items():
    print(f"- {emotion}: {score:.4f}")

top_emotion_index = torch.argmax(probs).item()
top_emotion = emotion_labels[top_emotion_index]

print(
    f"\nPredominant Emotion: {top_emotion} (Confidence: {emotion_scores[top_emotion]:.4f})"
)